# Importing libraries

In [ ]:
import os, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd

# Defining directories

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r"/different-diff/dataset/crate.io/"
analyze_dir = userhome + r"/different-diff/analyze/collect_metrics/analyze_crate.io/"

# Defining git command function

In [ ]:
def execute_command(cmd, work_dir):
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

In [ ]:
def execute_shell_command(id1, id2, cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return id1, id2, out, error
    pipe.wait()

# Extracting commit id

In [ ]:
commit_id = []
git_cmd = "git log --format=%H"
commit_id = str(execute_command(git_cmd, repository)).replace("b'",'').replace('b"','').replace('(','',1).split("\\n")[:-1]
commit_id

# Applying -w --ignore-blank-lines --diff-algorithm=histogram

In [ ]:
diffhist = []
for v in range(0, len(commit_id)-1):
    sys.stdout.write('\rExtracting differences between commits: %i' %(v+1) + ' out of %i' %(len(commit_id)-1))
    sys.stdout.flush()

    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=histogram " + commit_id[v+1] + " " + commit_id[v] + " --numstat"
    tmp = (str(execute_shell_command(commit_id[v] , commit_id[v+1], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffhist.append([str(tmp2)])
    
diffhistresult = []
for item in diffhist:
    if item == ', ':
        del item
    else:
        diffhistresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klhist = []
reshist = []
for v in range(0,len(diffhistresult)):
    for w in range(0,len(diffhistresult[v])):
        sys.stdout.write("\rProcessing {}/{}".format(v+1,len(diffhistresult)))
        sys.stdout.flush()
        klhist = diffhistresult[v][w].split(',')
        reshist.append(klhist)
print("\nProcess is complete")

In [ ]:
with open(analyze_dir + '02_diff-file/diffhist-metrics.csv', 'w') as csvfile:
    header = ['commit_id', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(reshist)):
        sys.stdout.write("\rProcessing {}/{}".format(m+1,len(reshist)))
        sys.stdout.flush()
        i = 2
        j = 3
        k = 4
        for x in range(0,int((len(reshist[m])-2)/3)):
            if (reshist[m][i]) == "-":
                pass
            else:
                line_changed = int(reshist[m][i]) + int(reshist[m][j])
                try:
                    if "{" in reshist[m][k]:
                        pattern = re.search(('({\w.*=>|{=>)'), reshist[m][k])
                    else:
                        pattern = re.search(('(\w.*=>)'), reshist[m][k])
                    pattern = pattern.groups()[0]
                    v = [reshist[m][0], reshist[m][1], reshist[m][i], reshist[m][j], reshist[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [reshist[m][0], reshist[m][1], reshist[m][i], reshist[m][j], reshist[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("\nThe file 'diffhist-metrics.csv' has been created")

In [ ]:
df_hist = pd.read_csv(analyze_dir + '02_diff-file/diffhist-metrics.csv', encoding='ISO-8859-1')
df_hist

# Applying -w --ignore-blank-lines --diff-algorithm=minimal

In [ ]:
diffmin = []
for v in range(0, len(commit_id)-1):
    sys.stdout.write('\rExtracting differences between commits: %i' %(v+1) + ' out of %i' %(len(commit_id)-1))
    sys.stdout.flush()

    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=minimal " + commit_id[v+1] + " " + commit_id[v] + " --numstat"
    tmp = (str(execute_shell_command(commit_id[v] , commit_id[v+1], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffmin.append([str(tmp2)])
    
diffminresult = []
for item in diffmin:
    if item == ', ':
        del item
    else:
        diffminresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klmin = []
resmin = []
for v in range(0,len(diffminresult)):
    for w in range(0,len(diffminresult[v])):
        sys.stdout.write("\rProcessing {}/{}".format(v+1,len(diffminresult)))
        sys.stdout.flush()
        klmin = diffminresult[v][w].split(',')
        resmin.append(klmin)
print("\nProcess is complete")

In [ ]:
with open(analyze_dir + '02_diff-file/diffmin-metrics.csv', 'w') as csvfile:
    header = ['commit_id', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmin)):
        sys.stdout.write("\rProcessing {}/{}".format(m+1,len(resmin)))
        sys.stdout.flush()
        i = 2
        j = 3
        k = 4
        for x in range(0,int((len(resmin[m])-2)/3)):
            if (resmin[m][i]) == "-":
                pass
            else:
                line_changed = int(resmin[m][i]) + int(resmin[m][j])
                try:
                    if "{" in resmin[m][k]:
                        pattern = re.search(('({\w.*=>|{=>)'), resmin[m][k])
                    else:
                        pattern = re.search(('(\w.*=>)'), resmin[m][k])
                    pattern = pattern.groups()[0]
                    v = [resmin[m][0], resmin[m][1], resmin[m][i], resmin[m][j], resmin[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [resmin[m][0], resmin[m][1], resmin[m][i], resmin[m][j], resmin[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("\nThe file 'diffmin-metrics.csv' has been created")

In [ ]:
df_min = pd.read_csv(analyze_dir + '02_diff-file/diffmin-metrics.csv', encoding='ISO-8859-1')
df_min

# Applying -w --ignore-blank-lines --diff-algorithm=myers

In [ ]:
diffmyers = []
for v in range(0, len(commit_id)-1):
    sys.stdout.write('\rExtracting differences between commits: %i' %(v+1) + ' out of %i' %(len(commit_id)-1))
    sys.stdout.flush()

    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=myers " + commit_id[v+1] + " " + commit_id[v] + " --numstat"
    tmp = (str(execute_shell_command(commit_id[v] , commit_id[v+1], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffmyers.append([str(tmp2)])
    
diffmyersresult = []
for item in diffmyers:
    if item == ', ':
        del item
    else:
        diffmyersresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klmyers = []
resmyers = []
for v in range(0,len(diffmyersresult)):
    for w in range(0,len(diffmyersresult[v])):
        sys.stdout.write("\rProcessing {}/{}".format(v+1,len(diffmyersresult)))
        sys.stdout.flush()
        klmyers = diffmyersresult[v][w].split(',')
        resmyers.append(klmyers)
print("\nProcess is complete")

In [ ]:
with open(analyze_dir + '02_diff-file/diffmyers-metrics.csv', 'w') as csvfile:
    header = ['commit_id', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmyers)):
        sys.stdout.write("\rProcessing {}/{}".format(m+1,len(resmyers)))
        sys.stdout.flush()
        i = 2
        j = 3
        k = 4
        for x in range(0,int((len(resmyers[m])-2)/3)):
            if (resmyers[m][i]) == "-":
                pass
            else:
                line_changed = int(resmyers[m][i]) + int(resmyers[m][j])
                try:
                    if "{" in resmyers[m][k]:
                        pattern = re.search(('({\w.*=>|{=>)'), resmyers[m][k])
                    else:
                        pattern = re.search(('(\w.*=>)'), resmyers[m][k])
                    pattern = pattern.groups()[0]
                    v = [resmyers[m][0], resmyers[m][1], resmyers[m][i], resmyers[m][j], resmyers[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [resmyers[m][0], resmyers[m][1], resmyers[m][i], resmyers[m][j], resmyers[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("\nThe file 'diffmyers-metrics.csv' has been created")

In [ ]:
df_myers = pd.read_csv(analyze_dir + '02_diff-file/diffmyers-metrics.csv', encoding='ISO-8859-1')
df_myers

# Applying -w --ignore-blank-lines --diff-algorithm=patience

In [ ]:
diffpat = []
for v in range(0, len(commit_id)-1):
    sys.stdout.write('\rExtracting differences between commits: %i' %(v+1) + ' out of %i' %(len(commit_id)-1))
    sys.stdout.flush()

    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=patience " + commit_id[v+1] + " " + commit_id[v] + " --numstat"
    tmp = (str(execute_shell_command(commit_id[v] , commit_id[v+1], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffpat.append([str(tmp2)])
    
diffpatresult = []
for item in diffpat:
    if item == ', ':
        del item
    else:
        diffpatresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klpat = []
respat = []
for v in range(0,len(diffpatresult)):
    for w in range(0,len(diffpatresult[v])):
        sys.stdout.write("\rProcessing {}/{}".format(v+1,len(diffpatresult)))
        sys.stdout.flush()
        klpat = diffpatresult[v][w].split(',')
        respat.append(klpat)
print("\nProcess is complete")

In [ ]:
with open(analyze_dir + '02_diff-file/diffpat-metrics.csv', 'w') as csvfile:
    header = ['commit_id', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(respat)):
        sys.stdout.write("\rProcessing {}/{}".format(m+1,len(respat)))
        sys.stdout.flush()
        i = 2
        j = 3
        k = 4
        for x in range(0,int((len(respat[m])-2)/3)):
            if (respat[m][i]) == "-":
                pass
            else:
                line_changed = int(respat[m][i]) + int(respat[m][j])
                try:
                    if "{" in respat[m][k]:
                        pattern = re.search(('({\w.*=>|{=>)'), respat[m][k])
                    else:
                        pattern = re.search(('(\w.*=>)'), respat[m][k])
                    pattern = pattern.groups()[0]
                    v = [respat[m][0], respat[m][1], respat[m][i], respat[m][j], respat[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [respat[m][0], respat[m][1], respat[m][i], respat[m][j], respat[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("\nThe file 'diffpat-metrics.csv' has been created")

In [ ]:
df_pat = pd.read_csv(analyze_dir + '02_diff-file/diffpat-metrics.csv', encoding='ISO-8859-1')
df_pat